In [1]:
import xgboost as xgb
import  numpy  as np

In [2]:
# 读取数据
data_train = xgb.DMatrix('./datasets/12.agaricus_train.txt')
data_test = xgb.DMatrix('./datasets/12.agaricus_test.txt')

[17:48:17] 6513x126 matrix with 143286 entries loaded from ./datasets/12.agaricus_train.txt
[17:48:17] 1611x126 matrix with 35442 entries loaded from ./datasets/12.agaricus_test.txt


In [6]:
# 设置参数
# eta  惩罚项
param = {'max_depth': 3, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}  # logistic
# param = {'max_depth': 3, 'eta': 0.3, 'silent': 1, 'objective': 'reg:logistic'}
watchlist = [(data_test, 'eval'), (data_train, 'train')]
n_round = 3
bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist)

# 计算错误率
y_hat = bst.predict(data_test)
y = data_test.get_label()
print(y_hat)
print(y)
error = sum(y != (y_hat > 0.5))
error_rate = float(error) / len(y_hat)
print('样本总数：\t', len(y_hat))
print('错误数目：\t%4d' % error)
print('错误率：\t%.5f%%' % (100 * error_rate))

[0]	eval-error:0.016139	train-error:0.014433
[1]	eval-error:0	train-error:0.001228
[2]	eval-error:0	train-error:0
[0.03906291 0.94524354 0.03906291 ... 0.9840406  0.01492236 0.9840406 ]
[0. 1. 0. ... 1. 0. 1.]
样本总数：	 1611
错误数目：	   0
错误率：	0.00000%


In [20]:
# 定义f: theta * x
def log_reg(y_hat, y):
    p = 1.0 / (1.0 + np.exp(-y_hat))
    g = p - y.get_label()
    h = p * (1.0 - p)
    return g, h


def error_rate(y_hat, y):
    return 'error', float(sum(y.get_label() != (y_hat > 0.5))) / len(y_hat)

In [21]:
# 自定义损失函数

# 设置参数
# eta  惩罚项
param = {'max_depth': 3, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}  
watchlist = [(data_test, 'eval'), (data_train, 'train')]
n_round = 3
# bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist)
bst = xgb.train(param, data_train, num_boost_round=n_round, evals=watchlist, obj=log_reg, feval=error_rate)

# 计算错误率
y_hat = bst.predict(data_test)
y = data_test.get_label()
print(y_hat)
print(y)
error = sum(y != (y_hat > 0.5))
error_rate = float(error) / len(y_hat)
print('样本总数：\t', len(y_hat))
print('错误数目：\t%4d' % error)
print('错误率：\t%.5f%%' % (100 * error_rate))

[0]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[1]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[2]	eval-error:0.016139	train-error:0.014433	eval-error:0.016139	train-error:0.014433
[0.00237227 0.7973376  0.00237227 ... 0.98565334 0.00125276 0.98565334]
[0. 1. 0. ... 1. 0. 1.]
样本总数：	 1611
错误数目：	  26
错误率：	1.61390%


In [22]:
param

{'max_depth': 3, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}